In [1]:
import cv2
import numpy as np

In [2]:
from keras.models import load_model

In [3]:
class_names = ['Shoes', 'Damage_Shoes']
width = 96
height = 96

In [4]:
model = load_model('shoe_cnn_model.h5')

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 94, 94, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 47, 47, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 47, 47, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 45, 45, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 22, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0

In [7]:
import time

#get references to the webcam,
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):
    
    #read new frame
    _, frame = camera.read()
    
    #flip frame
    frame = cv2.flip(frame, 1)
    
    #rescaling cam outout
    aspect = frame.shape[1] / float(frame.shape[0])
    res = int(aspect * camera_height)
    frame = cv2.resize(frame,(res,camera_height))
                    
    #add rectangle
    cv2.rectangle(frame,(300,75),(650,425),(240,100,0),2)
    
                    
    #get ROI
    roi = frame[75+2:425-2, 300+2:650-2]
    
    #parse BRG to RGB
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
                    
    #reseze to 224*224
    roi = cv2.resize(roi,(width,height))
   
    
    #predict
    roi_x = np.expand_dims(roi, axis=0)
                    
    predictions = model.predict(roi_x)
                    
    shoetype_1_pred, shoetype_2_pred = predictions[0]
                    
    #add text
    shoetype_1_text ='{}: {}%'.format(class_names[0], int(shoetype_1_pred*100))
    cv2.putText(frame, shoetype_1_text,(70,170),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6,(240,240,240),2)
    
    #add text
    shoetype_2_text ='{}: {}%'.format(class_names[1], int(shoetype_2_pred*100))
    cv2.putText(frame, shoetype_2_text,(70,200),
               cv2.FONT_HERSHEY_SIMPLEX, 0.6,(240,240,240),2)
                    
    #show the frame
    cv2.imshow("Test Out", frame)
    key = cv2.waitKey(1)
                    
    #quit cam pressing by 'q'
    if key & 0xFF == ord('q'):
        break
                    
camera.release()
cv2.destroyAllWindows()